In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

plt.style.use('dark_background')

# Read and clean data

In [2]:
df = pd.read_csv('../../../Data/Real Estate/Real Estate Data.csv', index_col=0, header=0)
df = df.reset_index(drop=True)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1404 entries, 0 to 1403
Data columns (total 68 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Type                      1404 non-null   object 
 1   Zoning Class              1404 non-null   object 
 2   Lot Frontage              1151 non-null   float64
 3   Lot Area                  1404 non-null   int64  
 4   Alley                     84 non-null     object 
 5   Lot Shape                 1404 non-null   object 
 6   Land Contour              1404 non-null   object 
 7   Lot Config                1404 non-null   object 
 8   Land Slope                1404 non-null   object 
 9   Nbhd                      1404 non-null   object 
 10  Location Condition        1404 non-null   object 
 11  Bldg Type                 1404 non-null   object 
 12  House Style               1404 non-null   object 
 13  OvQual                    1404 non-null   int64  
 14  Overall 

Type       Zoning Class  Lot Frontage  Lot Area Alley   
0  2-STORY 1946 & NEWER  Resid Low Density          65.0      8450   NaN  \
1  1-STORY 1946 & NEWER  Resid Low Density          80.0      9600   NaN   
2  2-STORY 1946 & NEWER  Resid Low Density          68.0     11250   NaN   
3  2-STORY 1945 & OLDER  Resid Low Density          60.0      9550   NaN   
4  2-STORY 1946 & NEWER  Resid Low Density          84.0     14260   NaN   

            Lot Shape Land Contour           Lot Config Land Slope   
0             Regular        Level           Inside lot     Gentle  \
1             Regular        Level  Frontage on 2 sides     Gentle   
2  Slightly irregular        Level           Inside lot     Gentle   
3  Slightly irregular        Level           Corner lot     Gentle   
4  Slightly irregular        Level  Frontage on 2 sides     Gentle   

            Nbhd  Location Condition          Bldg Type House Style  OvQual   
0  College Creek              Normal  1-family Detached     2 story       7  \
1        Veenker  Adjacent Feeder St  1-family Detached     1 story       6   
2  College Creek              Normal  1-family Detached     2 story       7   
3       Crawford              Normal  1-family Detached     2 story       7   
4     Northridge              Normal  1-family Detached     2 story       8   

   Overall Cond  Built  Year Remod Add Roof Style      Roof Material   
0             5   2003            2003      Gable  Composite Shingle  \
1             8   1976            1976      Gable  Composite Shingle   
2             5   2001            2002      Gable  Composite Shingle   
3             5   1915            1970      Gable  Composite Shingle   
4             5   2000            2000      Gable  Composite Shingle   

  Exterior Primary Masonry/Veneer  Masonry/Veneer Area Exterior Qual   
0     Vinyl Siding     Brick Face                196.0          Good  \
1     Metal Siding            NaN                  0.0       Average   
2     Vinyl Siding     Brick Face                162.0          Good   
3      Wood Siding            NaN                  0.0       Average   
4     Vinyl Siding     Brick Face                350.0          Good   

  Exterior Cond       Foundation   Basement Height Basement Cond   
0       Average  Poured Contrete     Good (90-99")       Average  \
1       Average     Cinder Block     Good (90-99")       Average   
2       Average  Poured Contrete     Good (90-99")       Average   
3       Average     Brick & Tile  Typical (80-89")          Good   
4       Average  Poured Contrete     Good (90-99")       Average   

  Basement Exposure       Basement Finish  Basement Finished Area   
0       No Exposure  Good Living Quarters                     706  \
1     Good Exposure   Avg Living Quarters                     978   
2      Min Exposure  Good Living Quarters                     486   
3       No Exposure   Avg Living Quarters                     216   
4      Avg Exposure  Good Living Quarters                     655   

   Basement Unfinished Area  Basement Area Heating Qual CentralAir   
0                       150            856    Excellent          Y  \
1                       284           1262    Excellent          Y   
2                       434            920    Excellent          Y   
3                       540            756         Good          Y   
4                       490           1145    Excellent          Y   

                          Electrical  1st Floor Area  2nd Floor Area   
0  Standard Circuit Breakers & Romex             856             854  \
1  Standard Circuit Breakers & Romex            1262               0   
2  Standard Circuit Breakers & Romex             920             866   
3  Standard Circuit Breakers & Romex             961             756   
4  Standard Circuit Breakers & Romex            1145            1053   

   Living Area Above Grade  Basement Full Baths  Basement Half baths   
0                     1710                    1   

### Rename columns

In [3]:
df = df.rename(columns = {'OvQual':'Overall Qual',
                          'Built':'Year',
                          'Garage Yr Built':'Year Garage'})

### Drop Alley column

In [4]:
df = df.drop(columns=['Alley'])

### Fill nans

In [5]:
df.isna().sum(axis=0).sort_values(ascending=False)

Masonry/Veneer              824
Lot Frontage                253
Year Garage                  63
Masonry/Veneer Area           8
Total Rooms Above Grade       0
Garage Type                   0
Fireplce Qual                 0
Fireplaces                    0
Functionality                 0
Type                          0
Garage Finish                 0
Kitchens Above Grade          0
Bedrooms Above Grade          0
Half Baths Above Grade        0
Full Baths Above Grade        0
Basement Half baths           0
Basement Full Baths           0
Kitchen Qual                  0
Garage Cars                   0
2nd Floor Area                0
Garage Area                   0
Garage Qual                   0
Garage Cond                   0
Paved Drive                   0
Wood Deck Area                0
Open Porch Area               0
Enclosed Porch Area           0
3 Season Porch Area           0
Screen Porch Area             0
Pool Area                     0
Pool Qual                     0
Fence   

In [6]:
gb1 = df.groupby(["Nbhd"])['Lot Frontage'].median().sort_values()

def fill_1(r):
    n = r.Nbhd
    
    x = r['Lot Frontage']
    
    if pd.isna(x):
        return gb1[n]
    else:
        return x
    # end
# end

df['Lot Frontage'] = df.apply(fill_1, axis=1)

In [7]:
gb2 = df.groupby(["Nbhd"])['Year Garage'].median().sort_values()

def fill_2(r):
    n = r.Nbhd
    
    x = r['Year Garage']
    
    if pd.isna(x):
        return gb2[n]
    else:
        return x
    # end
# end

df['Year Garage'] = df.apply(fill_2, axis=1)

In [8]:
gb3 = df.groupby(["Nbhd"])['Masonry/Veneer Area'].median().sort_values()

def fill_3(r):
    n = r.Nbhd
    
    x = r['Masonry/Veneer Area']
    
    if pd.isna(x):
        return gb2[n]
    else:
        return x
    # end
# end

df['Masonry/Veneer Area'] = df.apply(fill_3, axis=1)

In [9]:
df.isna().sum(axis=0).sort_values(ascending=False)

Masonry/Veneer              824
Type                          0
Kitchen Qual                  0
Garage Type                   0
Fireplce Qual                 0
Fireplaces                    0
Functionality                 0
Total Rooms Above Grade       0
Kitchens Above Grade          0
1st Floor Area                0
Bedrooms Above Grade          0
Half Baths Above Grade        0
Full Baths Above Grade        0
Basement Half baths           0
Basement Full Baths           0
Living Area Above Grade       0
Year Garage                   0
Garage Finish                 0
Garage Cars                   0
Garage Area                   0
Garage Qual                   0
Garage Cond                   0
Paved Drive                   0
Wood Deck Area                0
Open Porch Area               0
Enclosed Porch Area           0
3 Season Porch Area           0
Screen Porch Area             0
Pool Area                     0
Pool Qual                     0
Fence                         0
Sale Typ

### Drop Masonry/Veneer column

In [10]:
df = df.drop(columns=['Masonry/Veneer'])

# Feature engineering

### Floors

In [11]:
def get_floors(x):
    if x > 0:
        return 2
    else:
        return 1
    # end
# end

df['Floors'] = df['2nd Floor Area'].apply(get_floors)

### Areas

In [12]:
df['Indoor Area'] = df[
    [
        '1st Floor Area', '2nd Floor Area', 'Basement Area'
    ]
].sum(axis=1)

df['Outdoor Area'] = df[
    [
        '3 Season Porch Area', 'Enclosed Porch Area', 'Garage Area',
        'Open Porch Area', 'Pool Area', 'Screen Porch Area', 'Wood Deck Area'
    ]
].sum(axis=1)

df['Basement Finished Area Fraction'] = df['Basement Finished Area'] / df[['Basement Finished Area', 'Basement Unfinished Area']].sum(axis=1)
df['Basement Finished Area Fraction'] = df['Basement Finished Area Fraction'].fillna(0)

In [13]:
df = df.drop(
    columns=[
       '1st Floor Area', '2nd Floor Area', '3 Season Porch Area',
       'Basement Area', 'Basement Finished Area', 'Basement Unfinished Area',
       'Enclosed Porch Area', 'Garage Area', 'Living Area Above Grade',
       'Open Porch Area', 'Pool Area', 'Screen Porch Area', 'Wood Deck Area'
    ]
)

### Label encode qualities

In [14]:
from sklearn.preprocessing import OrdinalEncoder

# Define known categories
quality_order = ["Poor", "Fair", "Average", "Good", "Excellent"]

# Identify categorical columns that contain 'Average'
cols = df.columns[df.isin(['Average']).any(axis=0)]

# Replace unknown values with "Poor"
df[cols] = df[cols].applymap(lambda x: x if x in quality_order else "Poor")

# Initialize and fit OrdinalEncoder on a single-column DataFrame
encoder = OrdinalEncoder(
    categories=[quality_order]
)

encoder.fit(
    np.array(quality_order)[:,np.newaxis]
)

# Transform each column separately in a loop
for c in cols:
    df[c] = encoder.transform(df[[c]]).astype(int)[:,0]
# end

In [15]:
df['Basement Height'] = df['Basement Height'].apply(
    lambda x: x.split(" ")[0]
)

df['Basement Height'] = df['Basement Height'].apply(
    lambda x: x if x != "Typical" else "Average"
)

df['Basement Height'] = df['Basement Height'].apply(
    lambda x: x if x in quality_order else "Poor"
)

df['Basement Height'] = encoder.transform(df[['Basement Height']]).astype(int)[:,0]

In [16]:
df['Pool Qual'] = df['Pool Qual'].apply(
    lambda x: x if x in quality_order else "Poor"
)

df['Pool Qual'] = encoder.transform(df[['Pool Qual']]).astype(int)[:,0]

### Central Air

In [17]:
df['CentralAir'] = df['CentralAir'].replace(
    {
        'Y':1,
        'N':0
    }
)

df = df.rename(
    columns={
        'CentralAir':'Central Air'
    }
)

# Filter price outliers

In [18]:
a = df['Sale Price'].quantile(0.01)
b = df['Sale Price'].quantile(0.99)

df = df[ (a<df['Sale Price']) & (df['Sale Price']<b) ].reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1374 entries, 0 to 1373
Data columns (total 57 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Type                             1374 non-null   object 
 1   Zoning Class                     1374 non-null   object 
 2   Lot Frontage                     1374 non-null   float64
 3   Lot Area                         1374 non-null   int64  
 4   Lot Shape                        1374 non-null   object 
 5   Land Contour                     1374 non-null   object 
 6   Lot Config                       1374 non-null   object 
 7   Land Slope                       1374 non-null   object 
 8   Nbhd                             1374 non-null   object 
 9   Location Condition               1374 non-null   object 
 10  Bldg Type                        1374 non-null   object 
 11  House Style                      1374 non-null   object 
 12  Overall Qual        

# Preprocess data

In [19]:
X_raw = df.drop(columns=['Sale Price'])
y = df['Sale Price']

### Split numeric and categorical columns for separate processing

In [20]:
X_num = X_raw.select_dtypes(exclude=object)
X_cat = X_raw.select_dtypes(include=object)

### Process numerical data in several ways

In [21]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

X_num_ss = pd.DataFrame()
for c in X_num.columns:
    X_num_ss[c] = std_scaler.fit_transform(X_num[[c]])[:,0]
# end

### Get categorical dummies

In [22]:
X_dum = pd.DataFrame()
for c in X_cat.columns:
    dummy = pd.get_dummies(X_cat[c], prefix=c, dtype=int)
    dummy_sum = dummy.sum(axis=0)
    col_to_drop = dummy_sum.idxmax()
    dummy = dummy.drop(columns=[col_to_drop])
    X_dum = pd.concat([X_dum, dummy], axis=1)
# end
X_dum.head()

Type_1 STORY PUD  Type_1-1/2 STORY ALL AGES  Type_1-STORY 1945 & OLDER   
0                 0                          0                          0  \
1                 0                          0                          0   
2                 0                          0                          0   
3                 0                          0                          0   
4                 0                          0                          0   

   Type_1-STORY PUD  Type_2 FAMILY CONVERSION  Type_2-1/2 STORY ALL AGES   
0                 0                         0                          0  \
1                 0                         0                          0   
2                 0                         0                          0   
3                 0                         0                          0   
4                 0                         0                          0   

   Type_2-STORY 1945 & OLDER  Type_2-STORY 1946 & NEWER  Type_2-STORY PUD   
0                          0                          1                 0  \
1                          0                          0                 0   
2                          0                          1                 0   
3                          1                          0                 0   
4                          0                          1                 0   

   Type_DUPLEX  Type_SPLIT FOYER  Type_SPLIT OR MULTI-LEVEL   
0            0                 0                          0  \
1            0                 0                          0   
2            0                 0                          0   
3            0                 0                          0   
4            0                 0                          0   

   Zoning Class_Commercial  Zoning Class_Floating Village Resid   
0                        0                                    0  \
1                        0                                    0   
2                        0                                    0   
3                        0                                    0   
4                        0                                    0   

   Zoning Class_Resid High Density  Zoning Class_Resid Med Density   
0                                0                               0  \
1                                0                               0   
2                                0                               0   
3                                0                               0   
4                                0                               0   

   Lot Shape_Irregular  Lot Shape_Slightly irregular  Land Contour_Banked   
0                    0                             0                    0  \
1                    0                             0                    0   
2                    0                             1                    0   
3                    0                             1                    0   
4                    0                             1                    0   

   Land Contour_Depression  Land Contour_Hillside  Lot Config_Corner lot   
0                        0                      0                      0  \
1                        0                      0                      0   
2                        0                      0                      0   
3                        0                      0                      1   
4                        0                      0                      0   

   Lot Config_Cul-de-sac  Lot Config_Frontage on 2 sides   
0                      0                               0  \
1                      0                               1   
2                      0                               0   
3                      0                               0   
4                      0                               1   

   Lot Config_Frontage on 3 sides  Land Slope_Moderate  Land Slope_Severe   
0                               0                    0     

# Modeling

In [23]:
X = pd.concat([X_num_ss, X_dum], axis=1)

## Use parsimony to eliminate bad features first

In [24]:
from sklearn.ensemble import RandomForestRegressor

# Fit a Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)

# Get feature importances
importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
importances = importances[:100]

features_rf = importances.index

X_pars = X[ features_rf ]
importances

Indoor Area                                                                             0.512394
Overall Qual                                                                            0.265587
Year                                                                                    0.036638
Lot Area                                                                                0.016951
Outdoor Area                                                                            0.016819
Basement Finished Area Fraction                                                         0.014416
Year Remod Add                                                                          0.011371
Basement Height                                                                         0.010215
Lot Frontage                                                                            0.008870
Kitchen Qual                                                                            0.008826
Garage Cars                   

In [25]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

# Copy dataset to avoid modifying the original
X_vif = X_pars.copy()

# Iteratively remove features with high VIF
while True:
    # Add constant for intercept
    X_vif_with_const = sm.add_constant(X_vif)
    
    # Compute VIF for all features
    vif_series = pd.Series(
        [variance_inflation_factor(X_vif_with_const.values, i) for i in range(X_vif_with_const.shape[1])],
        index=X_vif_with_const.columns
    )
    
    # Exclude constant term and get the feature with the highest VIF
    vif_series = vif_series.drop('const', errors='ignore')
    highest_vif_feature = vif_series.idxmax()
    #display(vif_series)
    
    # Break the loop if all features have VIF ≤ 10
    if vif_series.loc[highest_vif_feature] <= 10:  # Checking the first feature after 'const'
        break

    # Drop the feature with the highest VIF
    X_vif = X_vif.drop(columns=[highest_vif_feature])

    print(f"Dropped: {highest_vif_feature} (VIF={vif_series.loc[highest_vif_feature]:.2f})")
# end

vif_series[:25]

Dropped: Sale Type_Home just constructed and sold (VIF=39.64)
Dropped: House Style_2 story (VIF=14.51)
Dropped: Type_1-1/2 STORY ALL AGES (VIF=11.22)
Dropped: Year (VIF=11.09)


Indoor Area                        4.950287
Overall Qual                       4.245326
Lot Area                           1.818257
Outdoor Area                       3.152417
Basement Finished Area Fraction    3.301054
Year Remod Add                     3.078135
Basement Height                    3.585613
Lot Frontage                       1.954380
Kitchen Qual                       2.783576
Garage Cars                        3.597237
Overall Cond                       2.046902
Fireplce Qual                      4.271340
Masonry/Veneer Area                1.373166
Year Garage                        4.407422
Fireplaces                         4.098903
Exterior Qual                      3.419080
Total Rooms Above Grade            4.584524
Full Baths Above Grade             3.285554
Land Contour_Banked                1.232802
Basement Full Baths                2.398744
Garage Type_Detached from home     2.174233
Bedrooms Above Grade               2.695982
Type_2-STORY 1946 & NEWER       

In [26]:
X = X.values
y = y.values

## Perform cross validation

In [30]:
from keras.models import Sequential
from keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay, PolynomialDecay
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def build_model( X_train, epochs, batch_size, decay_frac, initial_learning_rate, dropout_rate ):

    # Create model
    model = Sequential([
        Input(shape=X_train.shape[1]),
        Dense(20, activation='relu'),
        Dropout(dropout_rate),
        Dense(20, activation='relu'),
        Dropout(dropout_rate),
        Dense(20, activation='relu'),
        Dense(1, activation='relu'),
    ])
    
    # define training parameters
    decay_epochs    = int( epochs * decay_frac )
    n_train         = X_train.shape[0]
    steps_per_epoch = int( np.ceil( n_train / batch_size ) )
    decay_steps     = decay_epochs * steps_per_epoch
    
    lr_schedule_poly = PolynomialDecay(
        initial_learning_rate=initial_learning_rate,
        end_learning_rate=0.001,
        decay_steps=decay_steps,
        power=2.0
    )
    
    # Define the optimizer with a custom learning rate
    optimizer = Adam(
        learning_rate=lr_schedule_poly,
    )
    
    # Compile model
    model.compile(
        optimizer=optimizer,
        loss='mae'
    )

    return model
# end

In [31]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, r2_score

epochs                = 1000
batch_size            = 256
decay_frac            = 0.8
dropout_rate          = 0.25
initial_learning_rate = 0.01

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)

train_mae_list = []
test_mae_list = []
train_r2_list = []
test_r2_list = []

fold = 1
for train_index, test_index in kf.split(X):
    print(f"Fold {fold}/{n_splits}")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Build a fresh model (which resets optimizer state including the learning rate schedule)
    model = build_model(
        X_train,
        epochs,
        batch_size,
        decay_frac,
        initial_learning_rate,
        dropout_rate
    )

    # Early stopping callback
    early_stopping = EarlyStopping(
        monitor='val_loss',  # Monitor validation loss
        patience=100,          # Stop after 5 epochs without improvement
        restore_best_weights=True  # Restore the best weights after stopping
    )
    
    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, y_test),
        verbose=0,
        callbacks=[early_stopping]
    )
    
    # Evaluate on training data
    y_train_pred = model.predict(X_train, verbose=0)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    
    # Evaluate on test data
    y_test_pred = model.predict(X_test, verbose=0)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    print("MAE:")
    print(f"  Train: {train_mae:.4f}")
    print(f"  Test:  {test_mae:.4f}")
    print("R^2:")
    print(f"  Train: {train_r2:.4f}")
    print(f"  Test:  {test_r2:.4f}")
    print()
    
    train_mae_list.append(train_mae)
    test_mae_list.append(test_mae)
    train_r2_list.append(train_r2)
    test_r2_list.append(test_r2)
    
    fold += 1
# end

Fold 1/5
MAE:
  Train: 12278.7461
  Test:  18316.8711
R^2:
  Train: 0.9376
  Test:  0.7340

Fold 2/5
MAE:
  Train: 13816.1035
  Test:  16023.4473
R^2:
  Train: 0.8929
  Test:  0.9042

Fold 3/5
MAE:
  Train: 15627.8672
  Test:  18788.6836
R^2:
  Train: 0.8753
  Test:  0.8513

Fold 4/5
MAE:
  Train: 13237.2266
  Test:  19161.0469
R^2:
  Train: 0.9021
  Test:  0.7247

Fold 5/5
MAE:
  Train: 14780.8457
  Test:  17674.1797
R^2:
  Train: 0.8787
  Test:  0.8678



In [32]:
results_df = pd.DataFrame({
    'Train MAE': train_mae_list,
    'Test MAE': test_mae_list,
    'Train R2': train_r2_list,
    'Test R2': test_r2_list
})

results_df.describe()

Train MAE      Test MAE  Train R2   Test R2
count      5.000000      5.000000  5.000000  5.000000
mean   13948.157812  17992.845703  0.897321  0.816368
std     1306.195791   1233.199357  0.024981  0.081809
min    12278.746094  16023.447266  0.875289  0.724651
25%    13237.226562  17674.179688  0.878699  0.733969
50%    13816.103516  18316.871094  0.892941  0.851275
75%    14780.845703  18788.683594  0.902080  0.867791
max    15627.867188  19161.046875  0.937596  0.904155